# Homework 01-4 - 下載 Gossiping 板塊的內容

## Import the packages

載入 Pandas、Requests 和 BeautifulSoup。

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, Tag

## Extract articles

抽象一個 `extract_articles` 函式，擷取文章列表中的文章概要。

- 文章標題：使用 `div.title` 選取
- 文章作者：使用 `div.author` 選取
- 文章日期：使用 `div.date` 選取
- 連結：使用 `div.title` 的 `a` 子元素選取。因為連結是純 pathname，所以要補上 `https://www.ptt.cc` 網域。

In [2]:
def extract_articles(soup: BeautifulSoup) -> pd.DataFrame:
    articles = soup.find_all("div", class_="r-ent")
    results = []

    for i, article in enumerate(articles):
        assert isinstance(article, Tag)

        title = article.find("div", class_="title")
        author = article.find("div", class_="author")
        date = article.find("div", class_="date")
        link = article.select_one(".title > a")

        assert isinstance(title, Tag)
        assert isinstance(author, Tag)
        assert isinstance(date, Tag)

        if link is None:
            print(f"Article {i} has no link")
            continue

        assert isinstance(link, Tag)

        title_text = title.text.strip()
        author_text = author.text.strip()
        date_text = date.text.strip()
        link_text = link.get("href")

        assert isinstance(link_text, str)

        results.append({
            "文章標題": title_text,
            "作者": author_text,
            "文章日期": date_text,
            "文章連結": "https://www.ptt.cc" + link_text
        })

    results_df = pd.DataFrame(results)

    return results_df 

## Get previous page URL

抽象一個 `get_previous_page_url` 方法，從網頁中擷取「上一頁」的連結。

In [3]:
def get_previous_page_url(soup: BeautifulSoup) -> str:
    # 抓取 innerText 是 '‹ 上頁' 的元素
    prev_page_link = soup.find("a", string="‹ 上頁")
    assert isinstance(prev_page_link, Tag)

    prev_page_path = prev_page_link.get('href')
    assert isinstance(prev_page_path, str)

    return "https://www.ptt.cc" + prev_page_path

## Main application

取得 Gossiping 的最新一頁內容。

- 需要傳入 cookie `over18=1`，以免卡在驗證是否 18 歲的介面。
- 使用 `lxml` 取得比較快的解析速度。

In [4]:
# First Page

url = "https://www.ptt.cc/bbs/Gossiping/index.html"
resp = requests.get(url, headers={"cookie": "over18=1"})
soup = BeautifulSoup(resp.text, "lxml")

接著使用剛才的函式，取得最新一頁的文章。

In [5]:
pd_1 = extract_articles(soup)
pd_1

,文章標題,作者,文章日期,文章連結
0,[問卦] 戚夫人被做成人彘之後,hwsbetty,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401016....
1,[問卦] 畫三角形 外資大賣預言成真 接下來呢?,wwewcwwwf,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401161....
2,[問卦] 要出遠門 在阿嬤脖子上掛一個大餅可以嗎,poeta,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401218....
3,[爆卦] 台股史上五大跌點,ncc5566,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401296....
4,[問卦] 台股 前五大跌幅都在這兩年 各位有頭緒嗎,a5687920,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401354....
5,[問卦] 20元的雞蛋水餃股 會不會跌到下市？,ppp123,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401396....
6,[問卦] 吉卜力風格大家看膩了咩,Bastille,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401430....
7,Fw: [公告] 請留意新註冊帳號使用信件詐騙,ubcs,11/02,https://www.ptt.cc/bbs/Gossiping/M.1730554547....
8,[公告] 八卦板板規(2025.01.21),ubcs,1/21,https://www.ptt.cc/bbs/Gossiping/M.1737398137....
9,Fw: [公告] 請避免與登入1次之帳號進行交易 發錢,ubcs,3/12,https://www.ptt.cc/bbs/Gossiping/M.1741791500....


接著取得上一頁的 URL。

In [6]:
previous_page_url = get_previous_page_url(soup)
previous_page_url

'https://www.ptt.cc/bbs/Gossiping/index38953.html'

取得 Gossiping 的上一頁內容。

In [7]:
# Second Page
resp = requests.get(previous_page_url, headers={"cookie": "over18=1"})
soup = BeautifulSoup(resp.text, "lxml")

一樣進行解析。

In [8]:
pd_2 = extract_articles(soup)
pd_2

,文章標題,作者,文章日期,文章連結
0,Re: [問卦] 1天要吃8顆Tramadol的病情大概是有多嚴重,douge,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743399821....
1,[問卦] 不到千點！900點算鄉民小贏了嗎,a82k7,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743399839....
2,[新聞] 大白天在江南公寓殺害妻子的60多歲丈夫被,Qorqios,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743399899....
3,Re: [新聞] 天道盟公主開庭失控！亂扯一通還罵檢警,laptic,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743399974....
4,Re: [問卦] 不到千點！900點算鄉民小贏了嗎,gigaman,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743400105....
5,[新聞] 關稅戰引憂慮 川普要求玩更大,JQK2,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743400201....
6,[問卦] 在機場等飛機除了吃還能幹嘛,cj4385283,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743400218....
7,[新聞] 柯文哲「生理指數正常」無法留院！ 醫,WuSam,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743400252....
8,Re: [爆卦] 黑熊學院 疑似個資外洩,jknm0510a,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743400267....
9,[問卦] 有沒有☞【大民主義】☜的八卦？,Qorqios,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743400362....


合併兩頁的 dataframes。

In [9]:
# merge two pd
pd_all = pd.concat([pd_1, pd_2])
pd_all

,文章標題,作者,文章日期,文章連結
0,[問卦] 戚夫人被做成人彘之後,hwsbetty,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401016....
1,[問卦] 畫三角形 外資大賣預言成真 接下來呢?,wwewcwwwf,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401161....
2,[問卦] 要出遠門 在阿嬤脖子上掛一個大餅可以嗎,poeta,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401218....
3,[爆卦] 台股史上五大跌點,ncc5566,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401296....
4,[問卦] 台股 前五大跌幅都在這兩年 各位有頭緒嗎,a5687920,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401354....
5,[問卦] 20元的雞蛋水餃股 會不會跌到下市？,ppp123,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401396....
6,[問卦] 吉卜力風格大家看膩了咩,Bastille,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743401430....
7,Fw: [公告] 請留意新註冊帳號使用信件詐騙,ubcs,11/02,https://www.ptt.cc/bbs/Gossiping/M.1730554547....
8,[公告] 八卦板板規(2025.01.21),ubcs,1/21,https://www.ptt.cc/bbs/Gossiping/M.1737398137....
9,Fw: [公告] 請避免與登入1次之帳號進行交易 發錢,ubcs,3/12,https://www.ptt.cc/bbs/Gossiping/M.1741791500....


輸出為 `article-detail.csv`。

In [10]:
pd_all.to_csv("article-detail.csv", index=False)